
> Copyright 2023 Bruno César Brito Sant'Anna <mail@bsantanna.me>
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.



In [24]:
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.train.hyperdrive import (
  HyperDriveConfig,
  MedianStoppingPolicy,
  PrimaryMetricGoal,
  GridParameterSampling,
  choice)

In [25]:
# workspace settings
ws = Workspace.from_config()
ml_env = Environment.get(ws, name='AzureML-tensorflow-2.11-cuda11')
compute_target = ws.compute_targets['cpu-compute-cluster']

In [26]:
# experiment settings
script = ScriptRunConfig(
  source_directory='iban-validator-model',
  script='main-entrypoint.py',
  environment=ml_env,
  compute_target=compute_target
)

early_termination_policy = MedianStoppingPolicy(
  evaluation_interval=1, delay_evaluation=10
)
num_params = 4
ne = [500] * num_params
ia = ['relu'] * num_params
oa = ['softmax'] * num_params
op = ['adam'] * num_params
lo = ['categorical_crossentropy'] * num_params
param_sampling = GridParameterSampling({
  'batch-size': choice(4, 8, 16, 32),
  'num-epochs': choice(ne),
  'input-activation': choice(ia),
  'output-activation': choice(oa),
  'optimizer': choice(op),
  'loss': choice(lo),
})

hd_config = HyperDriveConfig(
  policy=early_termination_policy,
  run_config=script,
  hyperparameter_sampling=param_sampling,
  primary_metric_name="loss",
  primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
  max_total_runs=50,
  max_concurrent_runs=4
)

exp = Experiment(ws, 'iban-validator-model-experiment')

In [27]:
# experiment execution
run = exp.submit(hd_config)
print(run.get_portal_url())
run.wait_for_completion(show_output=True)

https://ml.azure.com/runs/HD_f70bd050-3847-4d40-9f16-64b347e2b5d2?wsid=/subscriptions/0d64c10a-a916-4b57-a110-e8573d336df4/resourcegroups/market-performance-eu-we/workspaces/mp-ml-workspace&tid=0de756ee-8c66-424e-89f7-7b0b1483eb80
RunId: HD_f70bd050-3847-4d40-9f16-64b347e2b5d2
Web View: https://ml.azure.com/runs/HD_f70bd050-3847-4d40-9f16-64b347e2b5d2?wsid=/subscriptions/0d64c10a-a916-4b57-a110-e8573d336df4/resourcegroups/market-performance-eu-we/workspaces/mp-ml-workspace&tid=0de756ee-8c66-424e-89f7-7b0b1483eb80

Streaming azureml-logs/hyperdrive.txt

[2023-07-15T20:06:10.410494][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-15T20:06:11.0354190Z][SCHEDULER][INFO]Scheduling job, id='HD_f70bd050-3847-4d40-9f16-64b347e2b5d2_1' 
[2023-07-15T20:06:10.9271469Z][SCHEDULER][INFO]Scheduling job, id='HD_f70bd050-3847-4d40-9f16-64b347e2b5d2_0' 
[2023-07-15T20:06:11.1383202Z][SCHEDULER][INFO]Scheduling job, id='HD_f70bd050-3847-4d40-9f16-64b347e2b5d2_2' 
[2023-0

{'runId': 'HD_f70bd050-3847-4d40-9f16-64b347e2b5d2',
 'target': 'cpu-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-07-15T20:06:09.663834Z',
 'endTimeUtc': '2023-07-15T20:10:12.986264Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"loss","goal":"minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '09c0fee2-1647-415d-b0a5-9c73969d9edd',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-76-generic-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.52.0',
  'space_size': '4096',
  'score': '0.0205930694937706',
  'best_child_run_id': 'HD_f70bd050-3847-4d40-9f16-64b347e2b5d2_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_f70bd050-3847-4d40-9f16-64b347e2b5d2_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'am